In [36]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid
from torchvision.transforms import ToTensor, Resize, Compose, CenterCrop, ColorJitter, Normalize, RandomRotation, RandomHorizontalFlip, RandomResizedCrop

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

In [37]:
class ConvolutionalNetwork(nn.Module):
    def __init__(self, dropout_rate=0.5):
        super(ConvolutionalNetwork, self).__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)   # Increase filter numbers
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)  # Additional conv layer
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1) # Additional conv layer
        
        # Fully connected layers
        self.fc1 = nn.Linear(256 * 7 * 7, 4096)      # Increase size and complexity
        self.fc2 = nn.Linear(4096, 1024)               # Additional fc layer
        self.fc3 = nn.Linear(1024, 256)                # Additional fc layer
        self.fc4 = nn.Linear(256, 102)                 # Output layer for 102 classes
        
        # Dropout layers
        self.dropout = nn.Dropout(p=dropout_rate)

    def forward(self, X):
        # Add more layers with relu activation and max pooling
        X = F.relu(self.conv1(X))
        X = F.max_pool2d(X, 2, 2)
        X = F.relu(self.conv2(X))
        X = F.max_pool2d(X, 2, 2)
        X = F.relu(self.conv3(X))
        X = F.max_pool2d(X, 2, 2)
        X = F.relu(self.conv4(X))
        X = F.max_pool2d(X, 2, 2)

        # Flatten the output for the fully connected layers
        X = X.view(-1, 256 * 7 * 7)

        # Apply relu activation and dropout to each fc layer
        X = F.relu(self.fc1(X))
        X = self.dropout(X)
        X = F.relu(self.fc2(X))
        X = self.dropout(X)
        X = F.relu(self.fc3(X))
        X = self.dropout(X)
        X = self.fc4(X)  # No activation needed here as nn.CrossEntropyLoss will be applied

        return X


In [38]:
# Define a transform to resize the images and convert them to tensors
transform = Compose([
    Resize((128, 128)),
    ToTensor()
])

# Load the dataset with the defined transform
dataset = datasets.Flowers102(root="/Users/maciek/cnn_data", split='train', download=True, transform=transform)
loader = DataLoader(dataset, batch_size=64, shuffle=False)

# Function to calculate mean and std
def calculate_mean_std(loader):
    mean = 0.
    std = 0.
    total_images_count = 0

    for (images, _) in loader:
        batch_samples = images.size(0)
        images = images.view(batch_samples, images.size(1), -1)
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)
        total_images_count += batch_samples

    mean /= total_images_count
    std /= total_images_count

    return mean, std

# Calculate mean and std
mean, std = calculate_mean_std(loader)
print(f"Mean: {mean}")
print(f"Std: {std}")

Mean: tensor([0.4330, 0.3819, 0.2964])
Std: tensor([0.2555, 0.2056, 0.2175])


In [39]:
train_transform = Compose([
    Resize((128, 128)),
    RandomRotation(degrees=30),
    RandomHorizontalFlip(p=0.5),
    RandomResizedCrop(size=(112, 112), scale=(0.8, 1.0)),
    ToTensor(),
    Normalize(mean=[0.4330, 0.3819, 0.2964], std=[0.2555, 0.2056, 0.2175])
])

val_transform = Compose([
    Resize((128, 128)),
    ToTensor(),
    Normalize(mean=[0.4330, 0.3819, 0.2964], std=[0.2555, 0.2056, 0.2175]),
])

test_transform = Compose([
    Resize((128, 128)),
    ToTensor(),
    Normalize(mean=[0.4330, 0.3819, 0.2964], std=[0.2555, 0.2056, 0.2175]),
])


In [40]:
    train_data = datasets.Flowers102(root="/Users/maciek/cnn_data", split='train', download=True, transform=train_transform)
    val_data = datasets.Flowers102(root="/Users/maciek/cnn_data", split='val', download=True, transform=val_transform)
    test_data = datasets.Flowers102(root="/Users/maciek/cnn_data", split='test', download=True, transform=test_transform)

In [41]:
    train_loader = DataLoader(train_data, batch_size=10, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=10, shuffle=False)
    test_loader = DataLoader(test_data, batch_size=10, shuffle=False)

In [42]:
    model = ConvolutionalNetwork()

In [43]:
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)

In [44]:
    device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

    model = model.to(device)
    criterion = criterion.to(device)

In [45]:
import time

start_time = time.time()

# Create Variables To Tracks Things
epochs = 100
train_losses = []
test_losses = []
train_correct = []
test_correct = []

# Early Stopping Parameters
patience = 5  # How many epochs to wait after val loss has stopped improving
min_val_loss = float('inf')  # Initialize to infinity
stale_epochs = 0  # Counter for epochs without improvement

# For Loop of Epochs
for i in range(epochs):
    trn_corr = 0
    tst_corr = 0

    # Train
    for b, (X_train, y_train) in enumerate(train_loader, 1):
        X_train, y_train = X_train.to(device), y_train.to(device)
        y_pred = model(X_train)  # get predicted values from the training set. Not flattened 2D
        loss = criterion(y_pred, y_train)  # how off are we? Compare the predictions to correct answers in y_train

        predicted = torch.max(y_pred.data, 1)[
            1]  # add up the number of correct predictions. Indexed off the first point
        batch_corr = (predicted == y_train).sum()  # how many we got correct from this batch. True = 1, False=0, sum those up
        trn_corr += batch_corr  # keep track as we go along in training.

        # Update our parameters
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

            # Print out some results
        if b % 10 == 0:
            print(f'Epoch: {i}  Batch: {b}  Loss: {loss.item()}')

    train_losses.append(loss)
    train_correct.append(trn_corr)

    # Test
    with torch.no_grad():
        val_loss = 0
        for b, (X_test, y_test) in enumerate(val_loader):
            X_test, y_test = X_test.to(device), y_test.to(device)
            y_val = model(X_test)
            predicted = torch.max(y_val.data, 1)[1]
            tst_corr += (predicted == y_test).sum()
            val_loss += criterion(y_val, y_test).item()  # Sum up the loss from each batch

    avg_val_loss = val_loss / len(val_loader)  # Calculate the average loss
    loss = criterion(y_val, y_test)

    test_losses.append(loss)
    test_correct.append(tst_corr)

    # Early Stopping Check
    if avg_val_loss < min_val_loss:
        min_val_loss = avg_val_loss
        stale_epochs = 0  # Reset the stale epochs counter
    else:
        stale_epochs += 1  # Increment the stale epochs counter
        if stale_epochs >= patience:
            print(f'Stopping early at epoch {i} due to overfitting.')
            break  # Break out of the loop

    print(f'Epoch: {i} Validation Loss: {avg_val_loss}')

current_time = time.time()
total = current_time - start_time
print(f'Training Took: {total / 60} minutes!')


Epoch: 0  Batch: 10  Loss: 4.654702663421631
Epoch: 0  Batch: 20  Loss: 4.632506370544434
Epoch: 0  Batch: 30  Loss: 4.638238430023193
Epoch: 0  Batch: 40  Loss: 4.615983009338379
Epoch: 0  Batch: 50  Loss: 4.642845153808594
Epoch: 0  Batch: 60  Loss: 4.634109020233154
Epoch: 0  Batch: 70  Loss: 4.631765365600586
Epoch: 0  Batch: 80  Loss: 4.63887882232666
Epoch: 0  Batch: 90  Loss: 4.622832298278809
Epoch: 0  Batch: 100  Loss: 4.618127346038818


RuntimeError: shape '[-1, 12544]' is invalid for input of size 163840

In [ ]:
# Convert GPU tensors to CPU tensors, detach them from the computation graph, and then to NumPy arrays
train_losses = [tl.cpu().detach().numpy() for tl in train_losses]
test_losses = [tl.cpu().detach().numpy() for tl in test_losses]

# Now you can plot using matplotlib
plt.plot(train_losses, label="Training Loss")
plt.plot(test_losses, label="Validation Loss")
plt.title("Loss at Epoch")
plt.legend()
plt.show(block=True)

In [ ]:
test_data

In [ ]:
plt.plot([t.cpu()/10 for t in train_correct], label="Training Accuracy")
plt.plot([t.cpu()/10 for t in test_correct], label="Validation Accuracy")
plt.title("Accuracy at the end of each Epoch")
plt.legend()

In [ ]:
#test_load_everything = DataLoader(test_data, batch_size=6149, shuffle=False)

In [ ]:
from torch.utils.data import DataLoader

# Use a smaller batch size
test_load_everything = DataLoader(test_data, batch_size=1024, shuffle=False)

correct = 0
total = 0
with torch.no_grad():
    for X_test, y_test in test_load_everything:
        X_test, y_test = X_test.to('mps'), y_test.to('mps')
        y_val = model(X_test)
        predicted = torch.max(y_val, 1)[1]
        correct += (predicted == y_test).sum().item()
        total += y_test.size(0)

# Calculate accuracy
accuracy = correct / total
print(f'Accuracy of the model on the test set: {accuracy * 100:.2f}%')

In [ ]:
#with torch.no_grad():
    #correct = 0
    #model.to('mps')  # Ensure the model is on the MPS device
    #for X_test, y_test in test_load_everything:
        #X_test = X_test.to('mps')  # Move X_test to MPS device
        #y_test = y_test.to('mps')  # Move y_test to MPS device as well
        #y_val = model(X_test)
        #predicted = torch.max(y_val, 1)[1]
        #correct += (predicted == y_test).sum()


In [ ]:
#X_test.shape

In [ ]:
#correct.item()/len(test_data)*100